# Setup & Imports

In [1]:
import logging
logging.basicConfig(format="%(levelname)s → %(message)s")

In [2]:
import random as rand
import math

# Functions

In [3]:
gcd = lambda a,b: b if(a==0) else gcd(b%a, a)

In [4]:
def square_and_multiply(num, power, n):
    if(power == 0):
        return 1

    log = logging.getLogger(square_and_multiply.__name__)
    log.setLevel(logging.WARNING)
    
    bin_digits = math.floor(math.log2(power))+1
    log.info(f"Binary Digits: {bin_digits}")

    mul = 1
    N = num
    mask = 1
    for i in range(bin_digits):
        if((mask&power) != 0):
            mul=(mul*N)%n

        # log.info(f"Mask: {mask}, N: {N}, mul: {mul}")
        N = (N**2)%n
        mask = mask<<1
    
    return mul
# square_and_multiply(2, 4, 100000)

In [5]:
def miller_rabin(num):
    assert(num != 1)
    if(num in [2, 3]): return 0
    if(num%2 == 0): return 1

    log = logging.getLogger(miller_rabin.__name__)
    log.setLevel(logging.WARNING)

    k = 0
    u = num-1
    while(u&1 != 1):
        u >>= 1
        k += 1

    u = int(u)
    log.info(f"u: {u}, k: {k}")

    a = rand.randrange(2, num-1)
    b = square_and_multiply(a, u, num)

    log.info(f"a: {a}, a^u mod n = {b}")

    if(b in [1, num-1]): return 0

    for i in range(k-1):

        b = square_and_multiply(b, 2, num)

        if(b == num-1): return 0
        if(b == 1): return 1

    return 1


def is_prime_mr(num, k):
    for i in range(k):
        if(miller_rabin(num)):
            return False
    return True

# is_prime_mr(1000000007, 10000)

In [6]:
def gen_prime(n_bits):
    
    log = logging.getLogger(gen_prime.__name__)
    log.setLevel(logging.WARNING)

    while True:
        num = (1<<(n_bits-1)) | 1
        num = num | rand.getrandbits(n_bits-2)

        assert(n_bits == math.floor(math.log2(num))+1)

        log.info(f"Candidate: {num}")

        if(is_prime_mr(num, 50)):
            log.info(f"Found: {num}")
            return num

# gen_prime(200)

# Setup

In [7]:
security_parameter = 32

In [8]:
def Setup(l, T):
    
    log = logging.getLogger(Setup.__name__)
    log.setLevel(logging.INFO)
    
    # Generate the group parameter
    p = gen_prime(2*security_parameter)
    q = gen_prime(2*security_parameter)
    
    n = p*q
    
    log.info(f"Setup Generated a group with: \n p: {p} \n q: {q} \n n: {n}")
    
    return (n, T, l)

# setup(30, 256)

# Eval

In [9]:
# Verifier sends a random L
L = gen_prime(security_parameter)

def Eval(pp, x):
    
    log = logging.getLogger(Eval.__name__)
    log.setLevel(logging.INFO)

    n, T, l = pp
    
    two_raised_to_T = 1<<T
    
    # Compute y = x^2^T
    y = square_and_multiply(x, two_raised_to_T, n)
    log.info(f"Evaluator computed y: {y}")
    
    log.info(f"Verifier Sent L: {L}")
    
    # Naively compute the quotient & The proof
    b = two_raised_to_T//L # <- Naively implemented
    pi = square_and_multiply(x, b, n)
    
    log.info(f"Prover computed pi: {pi}")
    
    return (y, pi)

# Verify

In [10]:
def Verify(pp, x, y, pi):
    
    log = logging.getLogger(Verify.__name__)
    log.setLevel(logging.INFO)
    
    n, T, l = pp
    
    # Compute r = 2^T mod n
    r = square_and_multiply(2, T, L)
    
    log.info(f"Verifier computed r: {r}")
    
    # Compute pi^L * x^r
    z = square_and_multiply(pi, L, n)
    log.info(f"pi^L: {z}")
    z = (z*square_and_multiply(x, r, n)) % n
    log.info(f"Verifier computed pi^L * x^r : {z}")
    
    return z == y

# Test

In [24]:
x = gen_prime(52)
pp = Setup(l=security_parameter, T=1<<20)
%time y, pi = Eval(pp, x)

INFO → Setup Generated a group with: 
 p: 10507806104026340093 
 q: 10757293600091565049 
 n: 113035555353845630294649975727906209557
INFO → Evaluator computed y: 24614558627845395267412264473974105114
INFO → Verifier Sent L: 2220704987
INFO → Prover computed pi: 89003931966429500331299559133528580568


CPU times: user 1min 15s, sys: 1.49 s, total: 1min 16s
Wall time: 1min 17s


In [25]:
%time Verify(pp, x, y, pi)

INFO → Verifier computed r: 1663314019
INFO → pi^L: 17488066454161422674793517885751548438
INFO → Verifier computed pi^L * x^r : 24614558627845395267412264473974105114


CPU times: user 2.41 ms, sys: 1.74 ms, total: 4.15 ms
Wall time: 2.75 ms


True

In [13]:
Verify(pp, x, y, pi-1)

INFO → Verifier computed r: 1088887513
INFO → pi^L: 16103303962012523264098061832089612216
INFO → Verifier computed pi^L * x^r : 104000144136516657666420338286666877965


False